In [ ]:
# Simple RNN Steps

# Sequential model -> Embedding layer -> SimpleRNN layer -> Dense layer -> Activation layer -> Optimizer -> Compile -> Fit -> Predict -> Loss Layer -> Metrics

In [3]:
import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')

print("Num GPUs Available: ", len(gpu_devices))

if len(gpu_devices) > 0:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

Num GPUs Available:  0


In [1]:
from tensorflow.keras.datasets import imdb

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
(25000,) (25000,) (25000,) (25000,)


In [8]:
from tensorflow.keras.preprocessing import sequence

X_train = sequence.pad_sequences(X_train, maxlen=500, padding='pre')
X_test = sequence.pad_sequences(X_test, maxlen=500, padding='pre')

In [9]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [13]:
model = Sequential()

model.add(Embedding(input_dim=10000, output_dim=32, input_length=500))
model.add(SimpleRNN(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

/Users/mac/Desktop/MLOps/Deep-Learning/venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [17]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping],
    validation_split=0.2
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 68ms/step - accuracy: 0.5638 - loss: 61645.1914 - val_accuracy: 0.7054 - val_loss: 0.5695
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 68ms/step - accuracy: 0.7641 - loss: 0.5065 - val_accuracy: 0.7386 - val_loss: 0.5197
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.8428 - loss: 0.3818 - val_accuracy: 0.7822 - val_loss: 0.4837
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 68ms/step - accuracy: 0.8588 - loss: 0.3380 - val_accuracy: 0.8004 - val_loss: 0.4575
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 69ms/step - accuracy: 0.9026 - loss: 0.2490 - val_accuracy: 0.8130 - val_loss: 0.4687
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 68ms/step - accuracy: 0.9228 - loss: 0.2115 - val_accuracy: 0.8134 - val_loss: 0.4570
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.9391 - loss: 0.1674 - val_accuracy: 0.7554 - val_loss: 0.5107
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 195s 312ms/step - accuracy: 0.9248 - loss: 0.2

In [18]:
model.save('movie_review_rnn_model.h5')

In [19]:
# Predict on test data

y_pred = model.predict(X_test)

y_pred

782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step


array([[0.19645496],
       [0.96923995],
       [0.9853167 ],
       ...,
       [0.17599806],
       [0.01241332],
       [0.7541486 ]], dtype=float32)

In [35]:
# Predict a data

data = "This movie was fantastic! The acting was great and the plot was thrilling."


encoded_data = [imdb.get_word_index().get(word, 2) + 3 for word in data]

data = sequence.pad_sequences([encoded_data], maxlen=500, padding='pre')

result = model.predict(data)

print(f"Sentiment: {'Positive' if result[0][0] > 0.5 else 'Negative'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Sentiment: Positive
